# Obligatory Exercise 2
##### John Isak F. Villanger

#### 1
- $k $ number of samples
- $2n $ number of literals
- $\epsilon $ maximal accepted error rate
- $\delta$ maximal accepted failure rate of $\epsilon$

The algorithm runs for $k$ iterations, generating an example and updates these variables:
- $Z_-$ The number of negative examples
- $Z_0(l): l \in literals$  The number of times the literal has been evaluated to 0 in the examples.
- $Z_{0+} (l): l \in literals$  The number of times the literal has been evaluated to 0 in the positive examples.

next the algorithm generates a hypothesis:

$$\frac{Z_0(l)}{k} \geq \frac{\epsilon}{64n^2}$$

where any literal over the threshold is included in the hypothesis. validity this term
is given in lemma 2 p363 of the learning from noisy examples paper.
We can see that literals are included here if they "rarely arise" from the probability distribution.
The intuition behind this is that there is little need to include positive literals that almost exclusively get
evaluated to 1, another way to put it: the formula $a\land b$ and the formula $b$ would be equivalent if $a$ always was true.

next the algorithm tries to remove from the hypothesis the literals that are often evaluated to 0 in positive examples.
by this threshold.

$$\frac{Z_{0+}}{Z_{0}} > \eta'+ \frac{\epsilon(1-2\eta_b)}{8n}$$

where $\eta'$ is an estimate of noise and $\eta_b$ is an upper bound of noise.
the validity of this term is given in lemma 7 p365 of the learning from noisy examples paper.


#### 2
An hypothesis class $H$ can shatter $N$ data points if we can find a hypothesis $h \in H$ that separates the positive examples from the negative for every problem.
VC dimension of an hypothesis class $H$ is the maximum number of data points that can be shattered by $H$

$VC(F) = \infty $. Since $h$ and $t$ are the same type(CNF Formula), we can always set $h = t$.
and this $h$ will separate every positive example from the negative examples.

If we assume $H$ is finite then $VC(F)\leq log_2\vert H\vert$.

Assume $VC(F)> log_2\vert H\vert$ and $VC(F) = n$
then there are $2^n$ possible ways to classify the data points as positive or negative.
$2^n > \vert H\vert \to log_2 2^n > log_2 \vert H\vert \to n > log_2 \vert H\vert \to VC(F)> log_2\vert H\vert$
Since $2^n > \vert H\vert$ By the reverse pigeonhole principle there must be a classification that the hypothesis space can't capture.
 The assumption $VC(F)> log_2\vert H\vert$ is incorrect.
so by contradiction $VC(F) \leq log_2\vert H\vert$

#### Some of my observations regarding the algorithm
The "width" of the distribution greatly seems to impact how well the algorithm works.
Wider distribution in this implementation will mean more often picking positive examples, while narrower will mean more
often picking negative examples. I think this means that it's hardest for the algorithm when positive examples are picked at a slightly rate than $\epsilon$,
since the hypothesis will very likely end up being inconsistent, but the error rate of an inconsistent hypothesis will be too high
to satisfy $\epsilon$. I find this problem arising with $sd = 15$ and the value of $k$ doesn't seem to impact it much, tested for $k = 10 000$.
I wonder if there is a mistake in my implementation causing this or if this is the type of scenario where are really high $k$ is needed.

I did not estimate $\eta_b$, the method presented in learning from noisy examples established a definitive upper bound,
but the runtime seems very high, but they also introduced more practical methods for estimating noise later on in the paper.
I believe a practical way to estimate $\eta_b$ could be to run the algorithm a set number of times, and pick the highest $\eta'$


In [1]:
import numpy as np
import random

class Literal:
    def __init__(self, value, name):
        self.value = value
        self.name = name
        self.in_target = False
        self.in_hypothesis = False
        self.negation = None
        #Number of times a literal is evaluated as 0 in an example
        self.z0 = 0
        #Number of times a literal is evaluated as 0 in a positive example
        self.z0_pos = 0
        #z0_pos / z0
        self.h = 0
    def __str__(self):
        return str(self.name) + "  " + str(self.value)

    def __repr__(self):
        return str(self.name) + " " + str(self.value)+", "


def generate_literals(n):
    literals = np.empty(2*n, dtype=Literal)
    for i in range(n):
        literals[2*i] = Literal(True, i)
        literals[2*i+1] = Literal(False, i)
        literals[2*i].negation = literals[2*i+1]
        literals[2*i+1].negation = literals[2*i]
        #print(literals[2*i])
        #print(literals[2*i+1])
    return literals


def generate_target(length, literals, distribution, noise, n):
    target = np.empty(length, dtype=Literal)
    for i in range(length):
        contradicting_literal = True
        while contradicting_literal:
            index = random.randint(0, n-1)
            value = distribution[index] + random.randint(-noise, noise)
            if not literals[2*index].in_target and not literals[2*index+1].in_target:
                if value < 50:
                    target[i] = literals[2*index+1]
                    target[i].in_target = True
                    contradicting_literal = False
                else:
                    target[i] = literals[2*index]
                    target[i].in_target = True
                    contradicting_literal = False
    return sorted(target, key= lambda x: x.name)


def generate_example(literals, distribution, noise, n, mean):
    example = np.empty(n, dtype=Literal)
    for i in range (0, n):
        value = distribution[i] + random.randint(-noise, noise)
        if value < mean:
            example[i] = literals[2*i+1]
        else:
            example[i] = literals[2*i]
    return example


def update_h(literals):
    for lit in literals:
        lit.h = lit.z0_pos/lit.z0 if lit.z0 != 0 else 1


def generate_distribution(size, mean, sd):
    #distribution = np.random.binomial(100, 0.5, size)
    distribution = np.random.normal(mean, sd, size)
    return distribution


def positive_example(example, noise_label):
    pos_example = True
    for i in range(example.size):
        if example[i].negation.in_target:
            pos_example = False
    val = random.random()
    return pos_example if val >= noise_label else not pos_example


def update_literals(example, label_example):
    for i in range(example.size):
        example[i].negation.z0 += 1
        if label_example:
            example[i].negation.z0_pos += 1


def update_hypothesis(literals, n_prime, sb):
    for literal in literals:
        if literal.in_hypothesis and literal.h > n_prime+sb/2:
            literal.in_hypothesis = False


def print_hypothesis(literals):
    for literal in literals:
        if literal.in_hypothesis:
            print(literal)


#Returns fraction of correct guesses by hypothesis
def estimate_accuracy(n_error_estimate, literals, distribution, noise, n, mean):
    correct_guesses = 0
    for i in range(n_error_estimate):
        example = generate_example(literals, distribution, noise, n, mean)
        correct_guess = True
        correct_guess_n = False
        pos_ex = positive_example(example, 0)
        for j in range(example.size):
            if example[j].negation.in_hypothesis:
                correct_guess = False
                correct_guess_n = True
        if (correct_guess and pos_ex) or (correct_guess_n and not pos_ex):
            correct_guesses += 1
    return correct_guesses/n_error_estimate

def generate_hypothesis(literals, k, q_i):
    for lit in literals:
        if lit.z0/k >= q_i/2:
            lit.in_hypothesis = True

def find_n2(literals):
    min_noise = 1
    for lit in literals:
        if lit.h is not None and lit.h < min_noise:
            min_noise = lit.h
    return min_noise

def train(distribution, literals, mean, n, noise_label, noise_literal, q_i, req_size, sb, z_neg):
    for j in range (req_size):
            example = generate_example(literals, distribution, noise_literal, n, mean)
            label_example = positive_example(example, noise_label)
            if not label_example:
                z_neg += 1
            update_literals(example, label_example)
    update_h(literals)
    n1 = z_neg/req_size
    generate_hypothesis(literals, req_size, q_i)
    n2 = find_n2(literals)
    #print("n1:",n1,"n2:",n2)
    n_prime = min(n1,n2)
    #print("Noise estimated to:", n_prime)
    #print("Hypothesis before final stage")
    #print_hypothesis(literals)
    #print("Hypothesis after final stage")
    update_hypothesis(literals, n_prime, sb)
    #print_hypothesis(literals)


#It picks a number from the Normal Distribution and assigns it to a literal
#If the number is more than the mean the probability of picking the positive literal increases in examples and target
#Noise is added so it will be a probability(instead of certainty) based on the normal distribution weather or not it picks the positive or negative literal
#Thus higher SD will mean less "noise"

def main():
    #I recommend using 50 as mean, and instead tweaking noise or SD in order to generate different types of data
    n = 10
    epsilon = 0.1
    delta = 0.3
    mean = 50
    sd = 15
    noise_literal = 10
    noise_label = 0.1
    target_size = 5
    error_greater_than_epsilon_count = 0

    noise_ub = 0.1

    k = 1
    q_h = epsilon / 2*(2*n)
    sb = q_h * (1-2*noise_ub)
    q_i = q_h/(8*(2*n))
    for i in range(3):
        print("testing with k =", k)
        for j in range (int(10/delta)):
            z_neg = 0
            literals = generate_literals(n)
            distribution = generate_distribution(n, mean, sd)
            #print(distribution)
            target = generate_target(target_size, literals,distribution, noise_literal, n)


            #req_size = int(2/(epsilon*(np.log(2)+np.log(1/delta))))
            #print("Required Size: ", req_size)
            #print(10/delta)
            #print("Distribution:\n",distribution)
            #print("Target:\n", target)
            train(distribution,literals,mean,n,noise_label,noise_literal,q_i,k,sb,z_neg)
            estimated_accuracy = estimate_accuracy(k*40, literals, distribution, noise_literal, n, mean)
            print("Accuracy:", estimated_accuracy)
            if (1 - estimated_accuracy) > epsilon:
                error_greater_than_epsilon_count += 1

        print("More than 10 higher than epsilon?", error_greater_than_epsilon_count)
        k *= 10
        error_greater_than_epsilon_count = 0

main()

testing with k = 1
Accuracy: 0.625
Accuracy: 0.75
Accuracy: 0.0
Accuracy: 0.125
Accuracy: 0.4
Accuracy: 0.825
Accuracy: 1.0
Accuracy: 0.35
Accuracy: 0.75
Accuracy: 0.8
Accuracy: 0.925
Accuracy: 0.625
Accuracy: 0.375
Accuracy: 0.725
Accuracy: 0.8
Accuracy: 0.6
Accuracy: 0.4
Accuracy: 0.5
Accuracy: 0.575
Accuracy: 0.875
Accuracy: 0.675
Accuracy: 0.475
Accuracy: 0.75
Accuracy: 0.6
Accuracy: 0.375
Accuracy: 0.975
Accuracy: 0.25
Accuracy: 0.9
Accuracy: 0.625
Accuracy: 0.125
Accuracy: 0.9
Accuracy: 0.75
Accuracy: 1.0
More than 10 higher than epsilon? 27
testing with k = 10
Accuracy: 0.9
Accuracy: 0.75
Accuracy: 0.5825
Accuracy: 0.765
Accuracy: 0.8525
Accuracy: 0.955
Accuracy: 0.9675
Accuracy: 0.895
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.94
Accuracy: 0.7675
Accuracy: 0.725
Accuracy: 0.88
Accuracy: 0.8675
Accuracy: 0.8125
Accuracy: 1.0
Accuracy: 0.955
Accuracy: 0.6725
Accuracy: 0.9725
Accuracy: 0.92
Accuracy: 0.77
Accuracy: 0.9775
Accuracy: 0.685
Accuracy: 1.0
Accuracy: 0.82
Accuracy: 1.0
Acc